# Оптическое распознавание символов

![Робот читает газету](./images/ocr.jpg)

Общей проблемой компьютерного зрения является обнаружение и интерпретация текста на изображении. Этот вид обработки часто называют *оптическим распознаванием символов* (optical character recognition, OCR).

## Чтение текста на изображении с помощью службы Computer Vision

Когнитивная служба **Computer Vision** обеспечивает поддержку OCR-задач, в том числе:

- API **для OCR**, который можно использовать для чтения текста на нескольких языках. Этот API используется асинхронно и может быть использован и для печатного, и для рукописного текста.
- API **для чтения**, оптимизированный для больших документов. Этот API может использоваться синхронно, и работает хорошо, когда нужно обнаружить и прочитать небольшое количество текста на изображении.

Эту службу можно использовать, создав ресурс либо **Computer Vision**, либо **Cognitive Services**.

Если вы еще этого не сделали, создайте ресурс **Cognitive Services** в своей подписке Azure.

> **Примечание**. Если у вас уже есть ресурс Cognitive Services, просто откройте его страницу **Быстрый запуск** и скопируйте его ключ и конечную точку в ячейку ниже. В противном случае следуйте приведенным ниже действиям для создания этого ресурса.

1. В другой вкладке браузера откройте портал Azure по адресу: https://portal.azure.com, войдя в систему под учетной записью Microsoft.

2. Нажмите кнопку **&#65291;Создать ресурс**, выполните поиск по строке *Cognitive Services* и создайте ресурс **Cognitive Services** со следующими настройками:
    - **Подписка**. *Ваша подписка Azure*.
    - **Группа ресурсов**. *Выберите или создайте группу ресурсов с уникальным именем*.
    - **Регион**. *Выберите любой доступный регион*:
    - **Имя**. *Введите уникальное имя*.
    - **Ценовая категория**. S0
    - **Подтверждаю, что прочитал и понял уведомления**. Выбрано.
3. Дождитесь завершения развертывания. Затем перейдите на свой ресурс Cognitive Services и на странице **Обзор** щелкните ссылку для управления ключами службы. Для подключения к вашему ресурсу когнитивных служб из клиентских приложений вам понадобятся конечная точка и ключи.

### Получение ключа и конечной точки для ресурса Cognitive Services

Для использования ресурса когнитивных служб клиентским приложениям необходимы их конечная точка и ключ аутентификации:

1. На портале Azure откройте страницу **Ключи и конечная точка** для вашего ресурса Cognitive Service, скопируйте **Ключ1** для вашего ресурса и вставьте его в приведенный ниже код, заменив подстановочный текст **YOUR_COG_KEY**.
2. Скопируйте **конечную точку** для своего ресурса и вставьте ее в код ниже, заменив **YOUR_COG_ENDPOINT**.
3. Выполните код в расположенной ниже ячейке с кодом, нажав на кнопку **Выполнить код в ячейке** (&#9655;) слева от ячейки.

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

Теперь, когда вы установили ключ и конечную точку, можно использовать ресурс службы компьютерного зрения для извлечения текста из изображения.

Давайте начнем с API **для OCR**, который позволит вам синхронно анализировать изображение и читать любой содержащийся в нем текст. В этом случае у вас есть рекламное изображение для вымышленной розничной компании Northwind Traders, которое содержит некоторый текст. Чтобы прочитать его, выполните код из ячейки ниже. 

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
%matplotlib inline

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Read the image file
image_path = os.path.join('data', 'ocr', 'advert.jpg')
image_stream = open(image_path, "rb")

# Use the Computer Vision service to find text in the image
read_results = computervision_client.recognize_printed_text_in_stream(image_stream)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)
plt.axis('off')
plt.imshow(img)

Найденный на изображении текст организован в иерархическую структуру из областей, строк и слов, и код считывает их для получения результатов.

В результатах можно увидеть текст, прочитанный из изображения. 

## Отображение ограничивающих прямоугольников

Результаты также включают координаты *ограничивающих прямоугольников* для строк текста и отдельных слов, найденных на изображении. Выполните код из ячейки ниже, чтобы увидеть координаты ограничивающих прямоугольников для строк текста на рекламном изображении, который вы получили ранее.

In [ ]:
# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Show the position of the line of text
        l,t,w,h = list(map(int, line.bounding_box.split(',')))
        draw.rectangle(((l,t), (l+w, t+h)), outline='magenta', width=5)

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Show the image with the text locations highlighted
plt.axis('off')
plt.imshow(img)

В результате на изображении будут показаны ограничивающие прямоугольники для каждой строки текста.

## Использование API для чтения

API для OCR, который вы использовали ранее, хорошо работает для изображений с небольшим количеством текста. Когда нужно прочитать большие объемы текста, например отсканированные документы, можно использовать API **для чтения**. Это требует многоступенчатого процесса:

1. Представьте изображение в службу Computer Vision для асинхронного чтения и анализа.
2. Дождитесь завершения операции анализа.
3. Получите результаты анализа.

Выполните код в следующей ячейке, чтобы использовать этот процесс для чтения текста в отсканированном письме менеджеру магазина Northwind Traders.

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'letter.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image to display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

Просмотрите результаты. Вот полная расшифровка письма, состоящая в основном из печатного текста с рукописной подписью. Оригинальное изображение письма отображается под результатами OCR (возможно, вам понадобится прокрутка, чтобы его увидеть).

## Чтение рукописного текста

В предыдущем примере в запросе на анализ изображения был задан режим распознавания текста, который оптимизировал работу с *печатным* текстом. Однако обратите внимание, что была прочитана и рукописная подпись.

Такая возможность чтения рукописного текста чрезвычайно полезна. Предположим, вы написали заметку, содержащую список покупок, и хотите использовать приложение на телефоне для прочтения заметки и расшифровки содержащегося в ней текста.

Выполните код из ячейки ниже, чтобы посмотреть пример операции чтения рукописного списка покупок.

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'note.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image to display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## Дополнительные сведения

Дополнительные сведения об использовании службы Computer Vision для распознавания текста см. в [документации по Computer Vision](https://docs.microsoft.com/ru-ru/azure/cognitive-services/computer-vision/concept-recognizing-text)